In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, UpSampling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# Convert class vectors to binary class matrices (one-hot encoding)
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Define the model
model = Sequential()

# Upsample images to at least 96x96x3
model.add(UpSampling2D(size=(3, 3), input_shape=(32, 32, 3)))

# Add MobileNetV2 model without the top layer (include_top=False)
model.add(MobileNetV2(weights='imagenet', include_top=False))

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add a fully connected layer with softmax activation for 10 classes
model.add(Dense(10, activation='softmax'))

# Freeze all layers in the base MobileNetV2 model
for layer in model.layers[1].layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model with the data augmentation generator
model.fit(x_train, y_train, batch_size=32,
          epochs=5,
          validation_data=(x_test, y_test))

# Evaluate the model
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


Epoch 1/5
1563/1563 [==============================] - 28s 16ms/step - loss: 1.2775 - accuracy: 0.5629 - val_loss: 1.1824 - val_accuracy: 0.5975
Epoch 2/5
1563/1563 [==============================] - 25s 16ms/step - loss: 1.0681 - accuracy: 0.6332 - val_loss: 1.0529 - val_accuracy: 0.6374
Epoch 3/5
1563/1563 [==============================] - 25s 16ms/step - loss: 1.0162 - accuracy: 0.6525 - val_loss: 0.9786 - val_accuracy: 0.6650
Epoch 4/5
1563/1563 [==============================] - 24s 15ms/step - loss: 0.9927 - accuracy: 0.6608 - val_loss: 1.0878 - val_accuracy: 0.6313
Epoch 5/5
313/313 [==============================] - 4s 12ms/step - loss: 1.0376 - accuracy: 0.6526
Test loss: 1.0375804901123047
Test accuracy: 0.6525999903678894


In [ ]:
# Suppose you decide to unfreeze the last 20 layers:
for layer in model.layers[1].layers[-20:]:
    layer.trainable = True

# Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with the data augmentation generator
model.fit(x_train, y_train, batch_size=32,
          epochs=5,
          validation_data=(x_test, y_test))



Epoch 1/5
1563/1563 [==============================] - 101s 21ms/step - loss: 1.1827 - accuracy: 0.6001 - val_loss: 0.9988 - val_accuracy: 0.6624
Epoch 2/5
1563/1563 [==============================] - 30s 19ms/step - loss: 0.8817 - accuracy: 0.6951 - val_loss: 0.8558 - val_accuracy: 0.7099
Epoch 3/5
1563/1563 [==============================] - 31s 20ms/step - loss: 0.7660 - accuracy: 0.7352 - val_loss: 0.7883 - val_accuracy: 0.7299
Epoch 4/5
1563/1563 [==============================] - 31s 20ms/step - loss: 0.6848 - accuracy: 0.7592 - val_loss: 0.7465 - val_accuracy: 0.7477
Epoch 5/5
1563/1563 [==============================] - 32s 20ms/step - loss: 0.6243 - accuracy: 0.7818 - val_loss: 0.7227 - val_accuracy: 0.7543


In [ ]:
# Train the model with the data augmentation generator
model.fit(x_train, y_train, batch_size=32,
          epochs=5,
          validation_data=(x_test, y_test))

Epoch 1/5
1563/1563 [==============================] - 30s 19ms/step - loss: 0.5742 - accuracy: 0.8003 - val_loss: 0.6970 - val_accuracy: 0.7657
Epoch 2/5
1563/1563 [==============================] - 31s 20ms/step - loss: 0.5307 - accuracy: 0.8131 - val_loss: 0.6816 - val_accuracy: 0.7702
Epoch 3/5
1563/1563 [==============================] - 32s 20ms/step - loss: 0.4924 - accuracy: 0.8267 - val_loss: 0.6695 - val_accuracy: 0.7763
Epoch 4/5
1563/1563 [==============================] - 30s 19ms/step - loss: 0.4582 - accuracy: 0.8398 - val_loss: 0.6633 - val_accuracy: 0.7788
Epoch 5/5
1563/1563 [==============================] - 31s 20ms/step - loss: 0.4299 - accuracy: 0.8500 - val_loss: 0.6539 - val_accuracy: 0.7822


In [ ]:
scores = model.evaluate(x_test, y_test, verbose=1)

313/313 [==============================] - 4s 12ms/step - loss: 0.6539 - accuracy: 0.7822


In [ ]:
scores

[0.65389484167099, 0.7821999788284302]